# Summary so far 

# TripAdvisor API - Best Option for Data Collection but needs to carefully monitored
Two different searches 
- Location search - returns the attractions in the query zone "New York City" 
- Then to get details on said attractions we must then search the locationiD to show indepth details on each location
### NB V Important - 1200 calls a day is lowest billing setting and only 5000 free calls a month before the begin to bill. Can become a problem if not properly managed

# Ticketmaster API - returns event data in NYC - provides sufficient info
- Good for Time Dates and Venues of live events and fututre event planning
- Not much good for attractions or depth of detail
###  5000 calls a day and no need for card details - solid option

# Google Places API - also a great option but very tedious
- Great depth of detail on each attraction in single search 
- Major downside only returns 20 rows of attractions per search.
### In order to use would need to send off lots of searches while slightly moving the coordinates about in order to attempt to collect lots of attractions. Billing is free and easily managed



## TripAdvisor API - 5000 free calls a month - cannot set lower than 1200 calls a day limit so MUST BE CAREFUL TO NOT EXCEED LIMIT AS MY CARD IS ON ACCOUNT

In [5]:
import requests 

API_KEY = "XXX"

# Base URL for TripAdvisor API search
SEARCH_URL = "https://api.content.tripadvisor.com/api/v1/location/search"

# Headers for the request
headers = {
    "accept": "application/json"
}

# Function to search for attractions in New York City
def search_attractions(query, limit=100):
    params = {
        "searchQuery": query,
        "language": "en",
        "key": API_KEY,
        "limit": limit  # Limit the number of results
    }
    
    response = requests.get(SEARCH_URL, headers=headers, params=params)
    response.raise_for_status()  # Check for HTTP errors
    data = response.json()
    
    # Extract basic information about each attraction
    attractions = [
        {
            "name": location.get("name"),
            "location_id": location.get("location_id"),
            "latitude": location.get("latitude"),
            "longitude": location.get("longitude"),
            "address": location.get("address_obj", {}).get("address_string"),
            "rating": location.get("rating"),
            "num_reviews": location.get("num_reviews")
        }
        for location in data.get("data", [])
    ]
    return attractions

# Fetch and print the first 100 attractions in New York City
attractions = search_attractions("New York City", 100)
for attraction in attractions:
    print(attraction)

{'name': 'New York City', 'location_id': '60763', 'latitude': None, 'longitude': None, 'address': 'New York City, NY', 'rating': None, 'num_reviews': None}
{'name': 'Motto by Hilton New York City Chelsea', 'location_id': '23448880', 'latitude': None, 'longitude': None, 'address': '113 West 24th Street, New York City, NY 10001', 'rating': None, 'num_reviews': None}
{'name': 'Arthouse Hotel New York City', 'location_id': '99288', 'latitude': None, 'longitude': None, 'address': '2178 Broadway at West 77th Street, New York City, NY 10024-6647', 'rating': None, 'num_reviews': None}
{'name': 'The Ludlow New York City', 'location_id': '6727310', 'latitude': None, 'longitude': None, 'address': '180 Ludlow St, New York City, NY 10002-1514', 'rating': None, 'num_reviews': None}
{'name': 'New York City Subway', 'location_id': '8130064', 'latitude': None, 'longitude': None, 'address': 'New York City, NY', 'rating': None, 'num_reviews': None}
{'name': 'Virgin Hotels New York City', 'location_id': '

In [8]:
url = "https://api.content.tripadvisor.com/api/v1/location/search?searchQuery=%22New%20York%20City%22&category=attractions&language=en&key=APIKEY"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{
  "data": [
    {
      "location_id": "8130064",
      "name": "New York City Subway",
      "address_obj": {
        "city": "New York City",
        "state": "New York",
        "country": "United States",
        "address_string": "New York City, NY"
      }
    },
    {
      "location_id": "555521",
      "name": "New York City Center",
      "address_obj": {
        "street1": "131 W 55th St",
        "street2": "Between 6th and 7th",
        "city": "New York City",
        "state": "New York",
        "country": "United States",
        "postalcode": "10019-5306",
        "address_string": "131 W 55th St Between 6th and 7th, New York City, NY 10019-5306"
      }
    },
    {
      "location_id": "555633",
      "name": "New York City Ballet",
      "address_obj": {
        "street1": "20 Lincoln Center Plz",
        "street2": "David H. Koch Theater, Lincoln Center for the Performing Arts complex",
        "city": "New York City",
        "state": "New York",
        "countr

In [9]:
len(response.text)

3875

In [14]:
import pandas as pd

API_KEY = "XXX"

# Base URL for TripAdvisor API search
SEARCH_URL = "https://api.content.tripadvisor.com/api/v1/location/search"

# Headers for the request
headers = {
    "accept": "application/json"
}

# Function to search for attractions in New York City and return as a DataFrame
def search_attractions(query, category, total_results=100):
    attractions = []
    offset = 0
    limit = 50  # Adjust based on the API's max limit per request

    while len(attractions) < total_results:
        params = {
            "searchQuery": query,
            "category": category,
            "language": "en",
            "key": API_KEY,
            "limit": limit,
            "offset": offset
        }

        response = requests.get(SEARCH_URL, headers=headers, params=params)
        data = response.json()

        if "data" not in data or not data["data"]:
            break

        for location in data["data"]:
            address_obj = location.get("address_obj", {})
            attractions.append({
                "name": location.get("name"),
                "location_id": location.get("location_id"),
                "latitude": location.get("latitude"),
                "longitude": location.get("longitude"),
                "street": address_obj.get("street1"),
                "city": address_obj.get("city"),
                "state": address_obj.get("state"),
                "country": address_obj.get("country"),
                "postalcode": address_obj.get("postalcode"),
                "address": address_obj.get("address_string"),
                "rating": location.get("rating"),
                "num_reviews": location.get("num_reviews")
            })

        offset += limit

    return pd.DataFrame(attractions[:total_results])

# Fetch the first 100 attractions in New York City
df_attractions = search_attractions("New York City", "attractions", 100)

# Print the DataFrame
print(df_attractions)

                               name location_id latitude longitude  \
0              New York City Subway     8130064     None      None   
1              New York City Center      555521     None      None   
2              New York City Ballet      555633     None      None   
3       Resorts World New York City     2423159     None      None   
4                New York City Hall    10031098     None      None   
..                              ...         ...      ...       ...   
95                     The Ride NYC     1951181     None      None   
96        New York City Fire Museum      110276     None      None   
97  Secret Food Tours New York City    12423267     None      None   
98         Helicopter New York City     4005106     None      None   
99      Babylon Tours New York City    11891707     None      None   

                         street           city     state        country  \
0                          None  New York City  New York  United States   
1        

In [15]:
# Print the DataFrame
df_attractions

,name,location_id,latitude,longitude,street,city,state,country,postalcode,address,rating,num_reviews
0,New York City Subway,8130064,None,None,None,New York City,New York,United States,None,"New York City, NY",None,None
1,New York City Center,555521,None,None,131 W 55th St,New York City,New York,United States,10019-5306,"131 W 55th St Between 6th and 7th, New York Ci...",None,None
2,New York City Ballet,555633,None,None,20 Lincoln Center Plz,New York City,New York,United States,10023-6913,"20 Lincoln Center Plz David H. Koch Theater, L...",None,None
3,Resorts World New York City,2423159,None,None,110-00 Rockaway Blvd,Jamaica,New York,United States,11420-1009,"110-00 Rockaway Blvd, Jamaica, NY 11420-1009",None,None
4,New York City Hall,10031098,None,None,Broadway and Murray St,New York City,New York,United States,10007,"Broadway and Murray St City Hall Park, New Yor...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
95,The Ride NYC,1951181,None,None,W 42nd St,New York City,New York,United States,10036,"W 42nd St, New York City, NY 10036",None,None
96,New York City Fire Museum,110276,None,None,278 Spring Street,New York City,New York,United States,10013-1405,"278 Spring Street, New York City, NY 10013-1405",None,None
97,Secret Food Tours New York City,12423267,None,None,323 6th Avenue,New York City,New York,United States,10014-4403,"323 6th Avenue, New York City, NY 10014-4403",None,None
98,Helicopter New York City,4005106,None,None,Downtown Manhattan Heliport,New York City,New York,United States,10004,Downtown Manhattan Heliport Pier 6 and the Eas...,None,None


In [16]:
#using the location ID for 8130064 - New York City Subway - can get additional info - lat and long is included
url = "https://api.content.tripadvisor.com/api/v1/location/8130064/details?language=en&currency=USD&key=APIKEY"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{
  "location_id": "8130064",
  "name": "New York City Subway",
  "web_url": "https://www.tripadvisor.com/Attraction_Review-g60763-d8130064-Reviews-New_York_City_Subway-New_York_City_New_York.html?m=66827",
  "address_obj": {
    "city": "New York City",
    "state": "New York",
    "country": "United States",
    "address_string": "New York City, NY"
  },
  "ancestors": [
    {
      "level": "City",
      "name": "New York City",
      "location_id": "60763"
    },
    {
      "abbrv": "NY",
      "level": "State",
      "name": "New York",
      "location_id": "28953"
    },
    {
      "level": "Country",
      "name": "United States",
      "location_id": "191"
    }
  ],
  "latitude": "40.75588",
  "longitude": "-73.98698",
  "timezone": "America/New_York",
  "website": "https://new.mta.info/",
  "write_review": "https://www.tripadvisor.com/UserReview-g60763-d8130064-New_York_City_Subway-New_York_City_New_York.html?m=66827",
  "ranking_data": {
    "geo_location_id": "60763",
   

In [36]:
API_KEY = 'XXX'
location_ids = ['8130064', '12423267', '4005106']  # Example list of location IDs

data = []

for location_id in location_ids:
    url = f"https://api.content.tripadvisor.com/api/v1/location/{location_id}/details?language=en&currency=USD&key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        location_data = response.json()
        # Extract required information
        location_info = {
            "location_id": location_data["location_id"],
            "name": location_data["name"],
            "latitude": location_data["latitude"],
            "longitude": location_data["longitude"],
            "website": location_data.get("website", None),
            "rating": location_data.get("rating", None),
            "ranking_string": location_data.get("ranking_data", {}).get("ranking_string", None),
            "weekday_text": location_data.get("hours", {}).get("weekday_text", None)
        }
        data.append(location_info)

# Create DataFrame
df = pd.DataFrame(data)

df

,location_id,name,latitude,longitude,website,rating,ranking_string,weekday_text
0,8130064,New York City Subway,40.75588,-73.98698,https://new.mta.info/,4.0,"#186 of 2,165 things to do in New York City","[Monday: 00:00 - 23:59, Tuesday: 00:00 - 23:59..."
1,12423267,Secret Food Tours New York City,40.73123,-74.00169,http://www.secretfoodtours.com/new-york/,5.0,#1 of 336 Food &amp; Drink in New York City,"[Monday: 11:00 - 23:00, Tuesday: 11:00 - 23:00..."
2,4005106,Helicopter New York City,40.70191,-74.01015,http://www.helicopternewyorkcity.com,4.5,"#297 of 1,511 Tours in New York City","[Monday: 09:00 - 17:00, Tuesday: 09:00 - 17:00..."


# TicketMaster API - 5000 free calls a day - no card on account needed

In [27]:
API_KEY = 'XXX'
url = 'https://app.ticketmaster.com/discovery/v2/events.json'
params = {
    'apikey': API_KEY,
    'city': 'New York',
    'classificationName': 'music',  # Adjust for different types of events
    'size': 100  # Number of events to retrieve
}

response = requests.get(url, params=params)

data = response.json()

events = []

if '_embedded' in data and 'events' in data['_embedded']:
    for event in data['_embedded']['events']:
        events.append({
            'name': event['name'],
            'date': event['dates']['start']['localDate'],
            'time': event['dates']['start'].get('localTime', 'TBA'),
            'venue': event['_embedded']['venues'][0]['name'],
            'city': event['_embedded']['venues'][0]['city']['name'],
            'state': event['_embedded']['venues'][0]['state']['name']
        })
events

[{'name': 'The The - Ensouled World Tour',
  'date': '2024-10-17',
  'time': '20:30:00',
  'venue': 'Beacon Theatre',
  'city': 'New York',
  'state': 'New York'},
 {'name': 'Little Shop of Horrors',
  'date': '2024-06-02',
  'time': '14:00:00',
  'venue': 'Westside Theatre Upstairs - NY',
  'city': 'New York',
  'state': 'New York'},
 {'name': 'Little Shop of Horrors',
  'date': '2024-06-09',
  'time': '14:00:00',
  'venue': 'Westside Theatre Upstairs - NY',
  'city': 'New York',
  'state': 'New York'},
 {'name': 'Little Shop of Horrors',
  'date': '2024-06-18',
  'time': '19:00:00',
  'venue': 'Westside Theatre Upstairs - NY',
  'city': 'New York',
  'state': 'New York'},
 {'name': 'Little Shop of Horrors',
  'date': '2024-07-12',
  'time': '20:00:00',
  'venue': 'Westside Theatre Upstairs - NY',
  'city': 'New York',
  'state': 'New York'},
 {'name': 'Little Shop of Horrors',
  'date': '2024-07-24',
  'time': '14:00:00',
  'venue': 'Westside Theatre Upstairs - NY',
  'city': 'New Yo

In [18]:
API_KEY = 'XXX'
url = 'https://app.ticketmaster.com/discovery/v2/events.json'
params = {
    'apikey': API_KEY,
    'city': 'New York',
    'classificationName': 'music',  # Adjust for different types of events
    'size': 100  # Number of events to retrieve
}

response = requests.get(url, params=params)

data = response.json()

events = []

if '_embedded' in data and 'events' in data['_embedded']:
    for event in data['_embedded']['events']:
        events.append({
            'name': event['name'],
            'date': event['dates']['start']['localDate'],
            'time': event['dates']['start'].get('localTime', 'TBA'),
            'venue': event['_embedded']['venues'][0]['name'],
            'city': event['_embedded']['venues'][0]['city']['name'],
            'state': event['_embedded']['venues'][0]['state']['name']
        })

df_events = pd.DataFrame(events)
print(df_events)

                             name        date      time  \
0   The The - Ensouled World Tour  2024-10-17  20:30:00   
1          Little Shop of Horrors  2024-06-02  14:00:00   
2          Little Shop of Horrors  2024-06-09  14:00:00   
3          Little Shop of Horrors  2024-06-18  19:00:00   
4          Little Shop of Horrors  2024-07-12  20:00:00   
..                            ...         ...       ...   
95         Little Shop of Horrors  2024-08-28  19:00:00   
96         Little Shop of Horrors  2024-08-13  19:00:00   
97         Little Shop of Horrors  2024-08-24  14:00:00   
98         Little Shop of Horrors  2024-08-21  14:00:00   
99         Little Shop of Horrors  2024-08-14  14:00:00   

                             venue      city     state  
0                   Beacon Theatre  New York  New York  
1   Westside Theatre Upstairs - NY  New York  New York  
2   Westside Theatre Upstairs - NY  New York  New York  
3   Westside Theatre Upstairs - NY  New York  New York  
4   We

In [19]:
df_events

,name,date,time,venue,city,state
0,The The - Ensouled World Tour,2024-10-17,20:30:00,Beacon Theatre,New York,New York
1,Little Shop of Horrors,2024-06-02,14:00:00,Westside Theatre Upstairs - NY,New York,New York
2,Little Shop of Horrors,2024-06-09,14:00:00,Westside Theatre Upstairs - NY,New York,New York
3,Little Shop of Horrors,2024-06-18,19:00:00,Westside Theatre Upstairs - NY,New York,New York
4,Little Shop of Horrors,2024-07-12,20:00:00,Westside Theatre Upstairs - NY,New York,New York
...,...,...,...,...,...,...
95,Little Shop of Horrors,2024-08-28,19:00:00,Westside Theatre Upstairs - NY,New York,New York
96,Little Shop of Horrors,2024-08-13,19:00:00,Westside Theatre Upstairs - NY,New York,New York
97,Little Shop of Horrors,2024-08-24,14:00:00,Westside Theatre Upstairs - NY,New York,New York
98,Little Shop of Horrors,2024-08-21,14:00:00,Westside Theatre Upstairs - NY,New York,New York


In [20]:
API_KEY = 'XXX'
url = 'https://app.ticketmaster.com/discovery/v2/events.json'
params = {
    'apikey': API_KEY,
    'city': 'New York',
}

response = requests.get(url, params=params)
data = response.json()

events = []

if '_embedded' in data and 'events' in data['_embedded']:
    for event in data['_embedded']['events']:
        events.append({
            'name': event['name'],
            'date': event['dates']['start']['localDate'],
            'time': event['dates']['start'].get('localTime', 'TBA'),
            'venue': event['_embedded']['venues'][0]['name'],
            'city': event['_embedded']['venues'][0]['city']['name'],
            'state': event['_embedded']['venues'][0]['state']['name']
        })

df_events_1 = pd.DataFrame(events)
df_events_1

,name,date,time,venue,city,state
0,Giants 100: A Night With Legends,2024-06-20,18:30:00,The Theater at MSG,New York,New York
1,The The - Ensouled World Tour,2024-10-17,20:30:00,Beacon Theatre,New York,New York
2,Harry Potter and the Cursed Child,2024-05-31,19:00:00,Lyric Theatre - NY,New York,New York
3,Harry Potter and the Cursed Child,2024-06-14,19:00:00,Lyric Theatre - NY,New York,New York
4,Harry Potter and the Cursed Child,2024-06-01,13:00:00,Lyric Theatre - NY,New York,New York
5,Harry Potter and the Cursed Child,2024-06-29,19:00:00,Lyric Theatre - NY,New York,New York
6,Harry Potter and the Cursed Child,2024-06-08,13:00:00,Lyric Theatre - NY,New York,New York
7,Harry Potter and the Cursed Child,2024-07-26,19:00:00,Lyric Theatre - NY,New York,New York
8,Harry Potter and the Cursed Child,2024-07-28,13:00:00,Lyric Theatre - NY,New York,New York
9,Harry Potter and the Cursed Child,2024-08-15,19:00:00,Lyric Theatre - NY,New York,New York


In [24]:
df_events_1['date'] = pd.to_datetime(df_events_1['date'])

# Sort events by 'date' in ascending order
df_events_1 = df_events_1.sort_values(by='date').reset_index(drop=True)

(df_events_1)

,name,date,time,venue,city,state
0,Harry Potter and the Cursed Child,2024-05-31,19:00:00,Lyric Theatre - NY,New York,New York
1,Harry Potter and the Cursed Child,2024-06-01,13:00:00,Lyric Theatre - NY,New York,New York
2,Harry Potter and the Cursed Child,2024-06-04,19:00:00,Lyric Theatre - NY,New York,New York
3,Harry Potter and the Cursed Child,2024-06-08,13:00:00,Lyric Theatre - NY,New York,New York
4,Harry Potter and the Cursed Child,2024-06-14,19:00:00,Lyric Theatre - NY,New York,New York
5,Harry Potter and the Cursed Child,2024-06-18,19:00:00,Lyric Theatre - NY,New York,New York
6,Giants 100: A Night With Legends,2024-06-20,18:30:00,The Theater at MSG,New York,New York
7,Harry Potter and the Cursed Child,2024-06-22,19:00:00,Lyric Theatre - NY,New York,New York
8,Harry Potter and the Cursed Child,2024-06-26,19:00:00,Lyric Theatre - NY,New York,New York
9,Harry Potter and the Cursed Child,2024-06-26,13:00:00,Lyric Theatre - NY,New York,New York


## Ticketmaster attraction Data not much use

In [31]:
url = "https://app.ticketmaster.com/discovery/v2/attractions.json"
params = {
    'apikey': 'XXX'
}

response = requests.get(url, params=params)
json_data = response.json()
print(json_data)

{'_embedded': {'attractions': [{'name': 'Brett Young', 'type': 'attraction', 'id': 'K8vZ9178Ee7', 'test': False, 'url': 'https://www.ticketmaster.com/brett-young-tickets/artist/1810116', 'locale': 'en-us', 'externalLinks': {'twitter': [{'url': 'https://twitter.com/BrettYoungMusic'}], 'itunes': [{'url': 'https://itunes.apple.com/us/artist/id1084576956'}], 'facebook': [{'url': 'https://www.facebook.com/brettyoungmusicpage'}], 'wiki': [{'url': 'https://en.wikipedia.org/wiki/Brett_Young_(singer)'}], 'instagram': [{'url': 'https://www.instagram.com/brettyoungmusic/'}, {'url': 'https://www.instagram.com/brettyoungmusic'}], 'musicbrainz': [{'id': '2ec5396e-4fe9-4918-867f-34370e012093'}], 'homepage': [{'url': 'http://brettyoungmusic.com/'}]}, 'images': [{'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/250/0a67289f-8424-47a0-950a-ce8c403c3250_TABLET_LANDSCAPE_3_2.jpg', 'width': 1024, 'height': 683, 'fallback': False}, {'ratio': '4_3', 'url': 'https://s1.ticketm.net/dam/a/250/0a67289f-8424-

In [34]:
attractions = json_data['_embedded']['attractions']
    

for attraction in attractions:
    print(attraction['name'])

Brett Young
Pete Davidson
Club 90s
Ben Howard
Isabel LaRosa
James Kennedy
10 Years
Magdalena Bay
Omid Djalili
Mehro
Barry Can't Swim
Jon Dore
Mindchatter
Been Stellar
Gaz Coombes
GINMANBLACHMANDAHL
Emil De Waal
P!NK
Phoenix Suns
PARTYNEXTDOOR


In [35]:
for attraction in attractions:
        name = attraction['name']
        genre = attraction['classifications'][0]['genre']['name']
        sub_genre = attraction['classifications'][0]['subGenre']['name']
        
        print("Name:", name)
        print("Genre:", genre)
        print("Sub Genre:", sub_genre)
        print()

Name: Brett Young
Genre: Country
Sub Genre: Country

Name: Pete Davidson
Genre: Comedy
Sub Genre: Comedy

Name: Club 90s
Genre: Pop
Sub Genre: Pop

Name: Ben Howard
Genre: Rock
Sub Genre: Pop

Name: Isabel LaRosa
Genre: Pop
Sub Genre: Pop

Name: James Kennedy
Genre: Dance/Electronic
Sub Genre: Dance/Electronic

Name: 10 Years
Genre: Rock
Sub Genre: Alternative Rock

Name: Magdalena Bay
Genre: Pop
Sub Genre: Pop

Name: Omid Djalili
Genre: Comedy
Sub Genre: Comedy

Name: Mehro
Genre: Pop
Sub Genre: Pop

Name: Barry Can't Swim
Genre: Dance/Electronic
Sub Genre: Dance/Electronic

Name: Jon Dore
Genre: Comedy
Sub Genre: Comedy

Name: Mindchatter
Genre: Dance/Electronic
Sub Genre: Dance/Electronic

Name: Been Stellar
Genre: Rock
Sub Genre: Rock

Name: Gaz Coombes
Genre: Rock
Sub Genre: Pop

Name: GINMANBLACHMANDAHL
Genre: Jazz
Sub Genre: Jazz

Name: Emil De Waal
Genre: Jazz
Sub Genre: Jazz

Name: P!NK
Genre: Rock
Sub Genre: Pop

Name: Phoenix Suns
Genre: Basketball
Sub Genre: NBA

Name: PART

# Google Places API is decent but only returns up to 60 results (3 pages of 20 results) per search so coordiantes will have to be changed over and over to cover whole area and find all attractions

In [37]:
API_KEY = 'XXX'
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Define parameters for the request
params = {
    'key': API_KEY,
    'location': '40.7128,-74.0060',  # New York City coordinates
    'radius': 5000,  # Radius in meters MAX 5000
    'type': 'tourist_attraction',  # Type of place to search for
    'keyword': 'attraction'  # Keyword to search for (optional)
}

# Send GET request 
response = requests.get(base_url, params=params)
results = response.json()

# Extract relevant information from the response
attractions = []
if 'results' in results:
    for place in results['results']:
        attraction = {
            'name': place.get('name', 'N/A'),
            'address': place.get('vicinity', 'N/A'),
             'keyword': 'attraction',
            'rating': place.get('rating', 'N/A'),
            
            'maxResultCount': 100  # Maximum number of results to retrieve / only returned 20 anyway
        }
        attractions.append(attraction)

# Print the list of attractions
for attraction in attractions:
    print(attraction)

{'name': 'Door to Nowhere', 'address': '156 W 42nd St, New York', 'rating': 4.6}
{'name': 'Trinity Place Skybridge', 'address': '1-11 Thames St, New York', 'rating': 4.7}
{'name': 'The Channel Gardens', 'address': 'New York', 'rating': 4.6}
{'name': 'Joey Ramone Place', 'address': '325 Bowery, New York', 'rating': 4.6}
{'name': 'Washington Square Fountain', 'address': 'Washington Square N, New York', 'rating': 4.7}
{'name': 'Each zone, Every One, Equal All', 'address': '200 W 42nd St, New York', 'rating': 3.7}
{'name': 'New York City Elevated View Point', 'address': '80 Furman St, Brooklyn', 'rating': 4.8}
{'name': 'Questing NY', 'address': '9 Maiden Ln, New York', 'rating': 5}
{'name': 'Manhattan Bridge Pedestrian Path', 'address': 'Brooklyn Heights', 'rating': 4.6}
{'name': 'Silverstein Family Park', 'address': 'Greenwich St, New York', 'rating': 4.5}
{'name': 'J Owen Grundy Park', 'address': 'Hudson St, Jersey City', 'rating': 4.7}
{'name': 'Morris Canal Park', 'address': '158 Washi

In [40]:
# Define API key and base URL
API_KEY = 'XXX'
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Define parameters for the request
params = {
    'key': API_KEY,
    'location': '40.7128,-74.0060',  # New York City coordinates
    'radius': 5000,  # Radius in meters MAX 5000
    'type': 'tourist_attraction',  # Type of place to search for
    'keyword': 'attraction',  # Keyword to search for (optional)
    'maxResultCount': 100  # Maximum number of results to retrieve
}

# Send GET request
response = requests.get(base_url, params=params)
results = response.json()

# Function to get place details by place ID
def get_place_details(place_id):
    details_url = 'https://maps.googleapis.com/maps/api/place/details/json'
    details_params = {
        'key': API_KEY,
        'place_id': place_id,
        'fields': 'name,formatted_address,opening_hours,rating,geometry'
    }
    details_response = requests.get(details_url, params=details_params)
    details_result = details_response.json()
    return details_result['result']

# Extract relevant information from the response
attractions = []
if 'results' in results:
    for place in results['results']:
        place_details = get_place_details(place['place_id'])
        attraction = {
            'name': place_details.get('name', 'N/A'),
            'address': place_details.get('formatted_address', 'N/A'),
            'rating': place_details.get('rating', 'N/A'),
            'opening_hours': place_details.get('opening_hours', {}).get('weekday_text', 'N/A'),
            'latitude': place_details.get('geometry', {}).get('location', {}).get('lat', 'N/A'),
            'longitude': place_details.get('geometry', {}).get('location', {}).get('lng', 'N/A')
        }
        attractions.append(attraction)

# Print the list of attractions
for attraction in attractions:
    print(attraction)

{'name': 'Door to Nowhere', 'address': '156 W 42nd St, New York, NY 10036, USA', 'rating': 4.6, 'opening_hours': ['Monday: Open 24 hours', 'Tuesday: Open 24 hours', 'Wednesday: Open 24 hours', 'Thursday: Open 24 hours', 'Friday: Open 24 hours', 'Saturday: Open 24 hours', 'Sunday: Open 24 hours'], 'latitude': 40.75563889999999, 'longitude': -73.9863542}
{'name': 'Trinity Place Skybridge', 'address': '1-11 Thames St, New York, NY 10006, USA', 'rating': 4.7, 'opening_hours': ['Monday: Open 24 hours', 'Tuesday: Open 24 hours', 'Wednesday: Open 24 hours', 'Thursday: Open 24 hours', 'Friday: Open 24 hours', 'Saturday: Open 24 hours', 'Sunday: Open 24 hours'], 'latitude': 40.7090585, 'longitude': -74.01152789999999}
{'name': 'The Channel Gardens', 'address': 'New York, NY 10020, USA', 'rating': 4.6, 'opening_hours': ['Monday: Open 24 hours', 'Tuesday: Open 24 hours', 'Wednesday: Open 24 hours', 'Thursday: Open 24 hours', 'Friday: Open 24 hours', 'Saturday: Open 24 hours', 'Sunday: Open 24 hou

In [48]:
attractions_df = pd.DataFrame(attractions)
attractions_df

,name,address,rating,opening_hours,latitude,longitude
0,Door to Nowhere,"156 W 42nd St, New York, NY 10036, USA",4.6,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",40.755639,-73.986354
1,Trinity Place Skybridge,"1-11 Thames St, New York, NY 10006, USA",4.7,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",40.709058,-74.011528
2,The Channel Gardens,"New York, NY 10020, USA",4.6,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",40.758501,-73.978087
3,Joey Ramone Place,"325 Bowery, New York, NY 10003, USA",4.6,N/A,40.725754,-73.991806
4,Washington Square Fountain,"Washington Square N, New York, NY 10012, USA",4.7,"[Monday: 6:00 AM – 1:00 AM, Tuesday: 6:00 AM –...",40.730851,-73.997431
5,"Each zone, Every One, Equal All","200 W 42nd St, New York, NY 10036, USA",3.7,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",40.755828,-73.986639
6,New York City Elevated View Point,"80 Furman St, Brooklyn, NY 11201, USA",4.8,N/A,40.700937,-73.996036
7,Questing NY,"9 Maiden Ln, New York, NY 10038, USA",5.0,"[Monday: 8:00 AM – 8:00 PM, Tuesday: 8:00 AM –...",40.709625,-74.009337
8,Manhattan Bridge Pedestrian Path,"Brooklyn Heights, NY 11201, USA",4.6,N/A,40.708806,-73.991655
9,Silverstein Family Park,"Greenwich St, New York, NY 10007, USA",4.5,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",40.713194,-74.011497
